In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt


from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import DisplayImage, DisplayHisto
from HOTS.Tools import SaveObject, LoadObject, GenerateHistogram
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.Monitor import DisplaySurface2D
from HOTS.Tools import EuclidianNorm, BattachaNorm, NormalizedNorm


In [3]:
tau = 0.02
R = 2
nb_cluster = [4,8,16]
ImageSize = (32,32)

This notebook show classification on Simple Alphabet database

# 1. Load Clustering, Training and Testing data

In [4]:
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(NbTrainingData=36, NbTestingData=40,
                                                        ClusteringData=[1, 32, 19, 22, 29],
                                                        OutOnePolarity = False, verbose=0, 
                                                        Path='../Data/ExtractedStabilized.mat', 
                                                        LabelPath='../Data/alphabet_label.pkl')


# 2. Defining the Network 

In [5]:
InputFilter = FilterNHBD(threshold=15,neighbourhood=2)
L1 = ClusteringLayer(tau=tau, R=R, verbose=1, ThrFilter=0, LearningAlgo='lagorce',
                         kernel='exponential',eta=None, eta_homeo=None, sigma=None)
L2 = ClusteringLayer(tau=10*tau, R=2*R, verbose=1, ThrFilter=0, LearningAlgo='lagorce'
                         ,kernel='exponential',eta=None, eta_homeo=None, sigma=None)
L3 = ClusteringLayer(tau=10*10*tau, R=2*2*R, verbose=1, ThrFilter=0, LearningAlgo='lagorce'
                         ,kernel='exponential',eta=None, eta_homeo=None, sigma=None)
Net = Network([InputFilter,L1,L2,L3])

# 3. Training the Cluster on 5 images

In [6]:
ClusterLayer, event_output = Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,
                                                     to_record=False, NbCycle=1)

Generation of SpatioTemporal Surface in ------ 1.09 s
Clustering SpatioTemporal Surface in ------ 1.80 s
Generation of SpatioTemporal Surface in ------ 1.43 s
Clustering SpatioTemporal Surface in ------ 2.32 s
Generation of SpatioTemporal Surface in ------ 3.31 s
Clustering SpatioTemporal Surface in ------ 7.61 s


# 4. Running the Network on training samples

In [7]:
event_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

Generation of SpatioTemporal Surface in ------ 7.66 s
Generation of SpatioTemporal Surface in ------ 12.54 s
Generation of SpatioTemporal Surface in ------ 26.70 s


# 5. Runing the Network on testing samples

In [8]:
event_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)

Generation of SpatioTemporal Surface in ------ 7.92 s
Generation of SpatioTemporal Surface in ------ 10.47 s
Generation of SpatioTemporal Surface in ------ 23.88 s


# 6. Classifying

In [9]:
Classif0 = Classifier(event_o_tr,event_o_te,TrainingLabel=label_tr, GroundTruth=label_te)
prediction, accuracy, method = Classif0.HistogramDistance(to_print=True)

battacha:0.85% ### normalized:0.825% ### euclidian:0.85% ### 
